In [18]:
import pandas as pd
import eform_query_processor as eqp
import os

In [19]:
# setting up the absolute savepath
save_path = r'C:\Users\aange\OneDrive\Desktop\Personal Python Projects\ferc_data_project\ferc_1_Q4_2021'

In [20]:
# loading in df containing files we want to grab
source_df = pd.read_csv('process_check/eform_form_1_2024-11-27_2021.csv')

source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        285 non-null    object
 1   Company    285 non-null    object
 2   Form       285 non-null    object
 3   Year       285 non-null    int64 
 4   Period     285 non-null    object
 5   Date/Time  285 non-null    object
 6   Status     285 non-null    object
 7   Filing ID  285 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 17.9+ KB


In [21]:
# checking the first 5
source_df_f5 = source_df[:5]

print(source_df_f5.info())

print(source_df_f5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        5 non-null      object
 1   Company    5 non-null      object
 2   Form       5 non-null      object
 3   Year       5 non-null      int64 
 4   Period     5 non-null      object
 5   Date/Time  5 non-null      object
 6   Status     5 non-null      object
 7   Filing ID  5 non-null      int64 
dtypes: int64(2), object(6)
memory usage: 452.0+ bytes
None
       CID                             Company    Form  Year Period  \
0  C000290          Duke Energy Carolinas, LLC  Form 1  2021     Q4   
1  C000822  Public Service Company of Colorado  Form 1  2021     Q4   
2  C000852       System Energy Resources, Inc.  Form 1  2021     Q4   
3  C000852       System Energy Resources, Inc.  Form 1  2021     Q4   
4  C011644     Pioneer Power and Light Company  Form 1  2021     Q4   

             Date/Time 

In [22]:
# create a list of filing id
filing_ids = list(source_df['Filing ID'])
print(filing_ids)

[158036, 146801, 146776, 135255, 135144, 113431, 112890, 99217, 98210, 98209, 96502, 96369, 96285, 96015, 96003, 96002, 95397, 94071, 94070, 94040, 94033, 93726, 93725, 93698, 93042, 92854, 92804, 92493, 91938, 91934, 91772, 91513, 91512, 91341, 91327, 91326, 91325, 91324, 91323, 91322, 91321, 91320, 91319, 91318, 91317, 91316, 91315, 91313, 91311, 91310, 91309, 91308, 91307, 91306, 91304, 91303, 91302, 91146, 91136, 91057, 91041, 90985, 90984, 90983, 90971, 90965, 90758, 90637, 90595, 90584, 90576, 90574, 90560, 90524, 90507, 90501, 90355, 90331, 90329, 90306, 90293, 90225, 90181, 90179, 90177, 90162, 90129, 90102, 90100, 90098, 90096, 90070, 90068, 90060, 90059, 90056, 90054, 90050, 90049, 90047, 90042, 90033, 90030, 90026, 90020, 90019, 90017, 90012, 90011, 90009, 90008, 90005, 90003, 89999, 89984, 89978, 89952, 89950, 89948, 89940, 89938, 89930, 89929, 89928, 89916, 89911, 89909, 89905, 89884, 89883, 89880, 89849, 89847, 89840, 89807, 89806, 89804, 89791, 89724, 79717, 79710, 79699

HERE WE ARE CREATING A FUNCTION THAT CAN TEST WHETHER OR NOT IT IS EASY TO PARALLELIZE FUNCTIONS

In [23]:
# return files
eqp.return_eform_filings(filing_ids=filing_ids, 
                         save_path=save_path)

Duke Energy Carolinas, LLC_ferc_714_Q42021_158036.xml


KeyboardInterrupt: 

Now we want to get a sense of how to sort our directories by time stamp
so that we can return a list of file paths to match with out pre existing
filing ids dataframe, allowing us to rename duplicate named files to assess
which files may have been revises or simply reuploaded at a later time.

In [7]:
# function to sort directory provided by either name or time
def sort_directory(sort_by: str, dir_path: str):

    '''
    In essence, first select whether you want to sort directory by name or by date created
    '''

    # purpose is to return the time the file was created
    def get_creation_time(item):
        item_path = os.path.join(dir_path, item)
        return os.path.getctime(item_path)
    
    # items meaning the files wihthin the directory path defined
    items = os.listdir(dir_path)

    if sort_by == 'date':
        '''
        in this case, user would like to sort directory path files by date
        '''
        print('Sorting files in path by creation date!')

        # sorting our items from oldest to newest
        sorted_items = sorted(items, key=get_creation_time)

        return sorted_items

    elif sort_by == 'name':
        '''
        in this case we would like to sort directory path files by name
        '''
        print('Sorting files in path by name!')

    else: 
        print('Please redefine how you would like to sort!')


In [28]:
# folder_path to test
fpath_714 = 'ferc_60_Q4_2021'

# attempt to check function
test_case = sort_directory(sort_by='date', dir_path=fpath_714)
print(test_case)


Sorting files in path by creation date!
['XBRL_Form_60.xbrl', 'XBRL_Form_60 (1).xbrl', 'XBRL_Form_60 (2).xbrl', 'XBRL_Form_60 (3).xbrl', 'XBRL_Form_60 (4).xbrl', 'XBRL_Form_60 (5).xbrl', 'XBRL_10_442_20220630061645.xbrl', 'wk-20211231.xml', 'wk-20211231 (1).xml', 'wk-20211231 (2).xml', 'wk-20211231 (3).xml', 'wk-20211231 (4).xml', 'wk-20211231 (5).xml', 'wk-20211231 (6).xml', 'wk-20211231 (7).xml', 'XBRL_10_31_20220426113527.xbrl', 'wk-20211231 (8).xml', 'wk-20211231 (9).xml', 'wk-20211231 (10).xml', 'wk-20211231 (11).xml', 'wk-20211231 (12).xml', 'wk-20211231 (13).xml', 'wk-20211231 (14).xml', 'wk-20211231 (15).xml', 'wk-20211231 (16).xml', 'wk-20211231 (17).xml', 'wk-20211231 (18).xml', 'wk-20211231 (19).xml', 'wk-20211231 (20).xml', 'wk-20211231 (21).xml', 'wk-20211231 (22).xml', 'wk-20211231 (23).xml', 'wk-20211231 (24).xml', 'wk-20211231 (25).xml', 'wk-20211231 (26).xml', 'wk-20211231 (27).xml', 'wk-20211231 (28).xml', 'wk-20211231 (29).xml', 'wk-20211231 (30).xml', 'wk-20211231 (

In [10]:
# function to change date times to the following format so that we can use it to rename files
# 12/9/2024, 4:25 PM --> 12_9_2024_4_25_PM 
# replace any [/], [,], [space] and [:] with an undescore

def clean_subnames(subname_list: list):
    '''
    given a string of a date or cname convert any defined chars to underscores
    '''
    # edited subname list
    new_subname_list = []

    for subname in subname_list:
        # first iteration checks for /
        iteration_1 = subname.replace("/", "_")
        iteration_2 = iteration_1.replace(",", "_")
        iteration_3 = iteration_2.replace(" ", "_")
        iteration_4 = iteration_3.replace(".", "_")
        final_iteration = iteration_4.replace(":", "_")
        new_subname_list.append(final_iteration)

    return new_subname_list


test_date_rename = clean_subnames(subname_list=["7/9/2024, 4:25 PM"])
cname_rename = clean_subnames(subname_list=['Southwest Power Pool, Inc.'])
print(test_date_rename)
print(cname_rename)

['7_9_2024__4_25_PM']
['Southwest_Power_Pool__Inc_']


In [11]:
# function aimed at setting up the renaming of files by grabbing info from df we used to grab files
def rename_xml_base(dframe: object):
    '''
    in essence, go through df and set up framework for the renaming of xml files grabbed
    '''
    # a finalized list of the renamed xml names
    renamed_xmls = []

    # columns to combine and combined strings list
    columns_to_combine = ['Form', 'Year', 'Company', 'Period', 'Date/Time', 'Filing ID']

    # combined strings
    for index, row in dframe.iterrows():

        # for each row in our dataframe we are combining select col values
        combined_string = ' '.join(row[columns_to_combine].astype(str))

        # in essence now changing all sub text into underscores
        final_string = clean_subnames(subname_list=list(combined_string))

        # combined final string where we have converted all sub elements in our words
        f_string = ''.join([str(item) for item in final_string])

        # printing combined string to see what's up
        renamed_xmls.append(f_string)
    
    return renamed_xmls



In [29]:
x = rename_xml_base(dframe=source_df)
print(x)


['Form_60_2021_PRODUCTION_TESTCOMPANY_7_Q4_5_24_2024__12_38_AM_180775', 'Form_60_2021_PRODUCTION_TESTCOMPANY_7_Q4_2_18_2024__11_19_PM_147023', 'Form_60_2021_PRODUCTION_TESTCOMPANY_7_Q4_2_17_2024__3_26_AM_147019', 'Form_60_2021_PRODUCTION_TESTCOMPANY_7_Q4_8_19_2023__3_18_AM_133642', 'Form_60_2021_PRODUCTION_TESTCOMPANY_7_Q4_4_1_2023__1_40_AM_98697', 'Form_60_2021_PRODUCTION_TESTCOMPANY_7_Q4_4_1_2023__12_00_AM_98696', 'Form_60_2021_Onward_Energy_Management__LLC_Q4_6_30_2022__9_42_AM_93660', 'Form_60_2021_Algonquin_Power_&_Utilities_Corp__Q4_6_2_2022__9_03_AM_92793', 'Form_60_2021_Liberty_Utilities_Service_Corp__Q4_6_1_2022__4_55_PM_92762', 'Form_60_2021_Liberty_Utilities_(Canada)_Corp__Q4_6_1_2022__4_50_PM_92760', 'Form_60_2021_Liberty_Energy_Utilities_(New_Hampshire)_Corp__Q4_5_9_2022__3_33_PM_90820', 'Form_60_2021_National_Grid_Service_Company_Inc__Q4_5_2_2022__3_50_PM_90611', 'Form_60_2021_National_Grid_Engineering_&_Survey__Inc__Q4_5_2_2022__3_10_PM_90609', 'Form_60_2021_CenterPoint_

okay, so now we have the tools to rename a file with a unique name that 
gives us information on who filed and exactly when, 

this makes it easy to parse through files that have been submitted multiple times
by specific entities....


now we want to work on 
(1) sorting the files within a directory
(2) iterating throuhg this file directory and renaming files with the processed
    file xml namees

we want to make sure we are accurately naming so we don't incorrecly name any files

In [13]:
# now, iterate through each file and rename it with what is in the list...
def rename_all_files(xml_df: object, dir_path: str, sort_by = 'date'):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name

    xml_df = df we used to pull the files from the ferc website
    sort_by = set to 'date' default but can also be set to 'name'
    dir_path = path of where the files downloaded is at....
    '''
    print('here is what we are working with: ')
    print(f'inputted df: {xml_df}')
    print(f'dir_path: {dir_path}')
    print('\n')

    # first, we are defining the directory path where we will be making the changes
    files_list = sort_directory(sort_by=sort_by, dir_path=dir_path)
    print(f'files list: {files_list}')

    # renaming xml files
    temp_xmls = rename_xml_base(dframe=xml_df)
    print(f'temp_xmls: {temp_xmls}')  

    # testing to see if things are matched 
    for file in files_list:
        print(f'{file}, {temp_xmls[files_list.index(str(file))]}')


In [14]:
# PLACEHOLDER
# now, iterate through each file and rename it with what is in the list...
def rename_all_files(xml_df: object, dir_path: str, sort_by = 'date'):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name

    xml_df = df we used to pull the files from the ferc website
    sort_by = set to 'date' default but can also be set to 'name'
    dir_path = path of where the files downloaded is at....
    '''
    print('here is what we are working with: ')
    print(f'inputted df: {xml_df}')
    print(f'dir_path: {dir_path}')
    print('\n')

    # first, we are defining the directory path where we will be making the changes
    files_list = sort_directory(sort_by=sort_by, dir_path=dir_path)
    print(files_list)

    # renaming xml files
    temp_xmls = rename_xml_base(dframe=xml_df)
    print(temp_xmls)  

    # now we want to iterate through each file and rename it
    for file in files_list: 
        # true file path
        true_file_path = f'{dir_path}/{file}'
        print(true_file_path)

        # new name
        print(f'new file name: {temp_xmls[file.index(str(file))]}')

        # rename file by grabbing location of the 'file' element
        new_fpath = os.rename(f'{true_file_path}', 
                              f'{dir_path}/{temp_xmls[files_list.index(str(file))]}.xml')




it looks like we need to split things between 
1. folder path
2. old file path
3. new file path

In [30]:
# folder_path to test
folder_path = 'ferc_60_Q4_2021'

rename_all_files(xml_df=source_df, dir_path=fpath_714)


here is what we are working with: 
inputted df:         CID                                         Company     Form  Year  \
0   C004085                        PRODUCTION TESTCOMPANY 7  Form 60  2021   
1   C004085                        PRODUCTION TESTCOMPANY 7  Form 60  2021   
2   C004085                        PRODUCTION TESTCOMPANY 7  Form 60  2021   
3   C004085                        PRODUCTION TESTCOMPANY 7  Form 60  2021   
4   C004085                        PRODUCTION TESTCOMPANY 7  Form 60  2021   
5   C004085                        PRODUCTION TESTCOMPANY 7  Form 60  2021   
6   C011570                   Onward Energy Management, LLC  Form 60  2021   
7   C011553               Algonquin Power & Utilities Corp.  Form 60  2021   
8   C011555                 Liberty Utilities Service Corp.  Form 60  2021   
9   C011554                Liberty Utilities (Canada) Corp.  Form 60  2021   
10  C011507  Liberty Energy Utilities (New Hampshire) Corp.  Form 60  2021   
11  C011360     